In [ ]:
import pandas as pd
import numpy as np
from backgammon import *
from pMCTS import *

In [ ]:
def player_chooser(player, board, rolls, legal_moves, depth, white_turn):

    if player == 'MCTS':

        return pUCT(board, rolls, legal_moves, depth)

    if player == 'Heuristic':

        return bestmove(legal_moves, white_turn)

    if player == 'Random':

        return random.choice(legal_moves)

In [ ]:
NUM = 2 # number of iterations of random games for NN data

def my_read_csv(): # this reads from the csv

    df = pd.read_csv('/content/drive/MyDrive/DS340/data.csv') # convert the csv into a dataframe
    boards = [] # create 3 arrays for each of the different columns
    wins = []
    total = []

    for i in range(len(df)): # go over the dataframe

        temp = df.iloc[i,0].split('.') # split the string into numbers
        temp = [int(i) for i in temp] # put those numbers into an array
        boards.append(temp) # add the array to array of boards
        wins.append(df.iloc[i,1])
        total.append(df.iloc[i,2])

    return boards, wins, total # return all three arrays


def data_creator(num = NUM): # this function augments existing data in the csv with another num simulations of the game

    boards, wins, total = my_read_csv() # the csv stores the board state, the number of times white won from that state, the total games that this board state came up in 
    trainer = dict() # this is the dictionary that will store the existing results

    for i in range(len(boards)): # go over the csv data and throw it in the dictionary

        board = boards[i]
        str_rep = '' # this will hold the string representation of the board state
        for item in board:

            str_rep += '.' + str(item)
        trainer[str_rep[1:]] = [wins[i], total[i]]

    for i in range(num): # do the simulations
        boards, winner = data_play('MCTS', 'Random', 5) # choose which types of AI you want to see play

        for board in boards: # go over the boards created in the process

            str_rep = '' # this will hold the string representation
            for item in board:

                str_rep += '.' + str(item)

            existing = trainer.get(str_rep[1:], [0,0]) # see if this board already exists
            existing[0] += winner # add the information of wins
            existing[1] += 1
            trainer[str_rep[1:]] = existing # put the information back

    df = pd.DataFrame.from_dict(data = trainer, orient = 'index') # turn this into a dataframe

    df.to_csv('data.csv') # converty the dataframe into a csv
    !cp data.csv "drive/My Drive/" # upload the csv


def convert(): # this converts the csv into two arrays of board states and wins 

    boards, wins, total = my_read_csv()
    wins = np.asarray(wins)
    total = np.asarray(total)
    wins = wins/total

    return boards, wins

boards, wins = convert()

In [ ]:
data_creator(10000)

In [ ]:
"""function that turns standard board states into boards with feature-extraction 
to emphasize the significance of stacking pieces, and clarify the distinction between
positive and negative numbers as black/white pieces to the AI 
"""
def transform_board(board):
    board = list(board)
    game_state = [board[0]] + board[25:]

    board_states = np.zeros((24, 4, 1))
    for ind, val in enumerate(board[BOARD_SPLICE]): # segregates each item on the board based on the number of pieces on that space
        if val > 1:
            board_states[ind][0][0] = val
        elif val == 1:
            board_states[ind][1][0] = val
        elif val == -1:
            board_states[ind][2][0] = val
        else:
            board_states[ind][3][0] = val

    return board_states, game_state

In [ ]:
#modifies default board states into states useable by CNN
mod_boards, states, y = [], [], []
for ind, board in enumerate(boards):
    x1, x2 = transform_board(board)
    mod_boards.append(x1)
    states.append(x2)
    y.append(wins[ind])

mod_boards = np.asarray(mod_boards)
states = np.asarray(states)
y = np.asarray(y)

In [0]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Conv2D, ReLU, Dropout, Flatten, Dense, concatenate

#Framework for CNN model
board = Input((24, 4, 1))
play_state = Input((5,))

conv_1 = Conv2D(32, (12,2),strides=(1,2), activation="relu")(board)
drop_1 = Dropout(0.2)(conv_1)
conv_2 = Conv2D(64, kernel_size=(2,2), activation="relu")(drop_1)
drop_2 = Dropout(0.2)(conv_2)
flattened = Flatten()(drop_2)

dense_1 = Dense(128, activation="relu")(flattened)
dropout_1 = Dropout(0.2)(dense_1)

dense_2 = Dense(64, activation="relu")(dropout_1)
dropout_2 = Dropout(0.2)(dense_2)

dense_3 = Dense(8, activation="relu")(dropout_2)
dropout_3 = Dropout(0.2)(dense_3)

merged = concatenate([dropout_3, play_state])

output = Dense(1, activation="sigmoid")(merged)
model = keras.models.Model(inputs=(board,play_state), outputs=output)

model.compile(optimizer="adam", loss="mean_squared_error")

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Conv2D, ReLU, Dropout, Flatten, Dense, concatenate

#Framework for fully connected network
input = Input((29,))

dense_1 = Dense(128, activation="relu")(input)
dropout_1 = Dropout(0.2)(dense_1)

dense_2 = Dense(64, activation="relu")(dropout_1)
dropout_2 = Dropout(0.2)(dense_2)

dense_3 = Dense(8, activation="relu")(dropout_2)
dropout_3 = Dropout(0.2)(dense_3)

output = Dense(1, activation="sigmoid")(dropout_3)
model = keras.models.Model(inputs=(input), outputs=output)

model.compile(optimizer="adam", loss="mean_squared_error")

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/DS340/BackgammonNN') # this is the NN model that we trained and saved in

model.summary()

In [ ]:
from keras.callbacks import EarlyStopping
earlyStop = EarlyStopping("val_loss",patience=4)
model.fit(x=[mod_boards,states],y=y,epochs=30, validation_split=0.1,callbacks=[earlyStop])

In [ ]:
def NN_rollout(board, model = model):
    board, state = transform_board(board)
    b = np.asarray([board])
    s = np.asarray([state])
    pred = model.predict([b,s], verbose=0)
    if node.board[WHITE_TURN] == 1:
        pred = 1-pred
    return pred